In [1]:
import mlflow 

In [2]:
mlflow.set_tracking_uri("http://ec2-13-61-146-35.eu-north-1.compute.amazonaws.com:5000/")

In [3]:
mlflow.set_experiment("Boosting")

<Experiment: artifact_location='s3://my-mlflow-bucket-123/581382709887216997', creation_time=1755545838660, experiment_id='581382709887216997', last_update_time=1755545838660, lifecycle_stage='active', name='Boosting', tags={}>

In [4]:
import pandas as pd

In [5]:
df = pd.read_csv("dataset.csv")

In [6]:
df.shape

(39574, 5)

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from xgboost import XGBClassifier
import optuna
import mlflow.sklearn
from lightgbm import LGBMClassifier


c:\Users\iamha\OneDrive\Documents\SentimentAnalysis\senti\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
ngram_range = (1, 1)
max_features = 8000

x_train, x_test, y_train, y_test = train_test_split(df['text'], df['label'], random_state=42, test_size=0.2)

vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
x_train_vec = vectorizer.fit_transform(x_train)
x_test_vec = vectorizer.transform(x_test)

def log_mlflow(model_name, model, x_train, x_test, y_train, y_test):
    with mlflow.start_run():
        mlflow.set_tag("mlflow.runName", f"{model_name}_TF-IDF")
        mlflow.set_tag("experiment_type", "algo_comparison")

        mlflow.log_param("algo_name", model_name)

        model.fit(x_train, y_train)
        y_pred = model.predict(x_test)

        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        mlflow.sklearn.log_model(model, f"{model_name}")

def objective_xgboost(trial):
    n_estimators = trial.suggest_int("n_estimators", 50, 300)
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-1, log=True)
    max_depth = trial.suggest_int("max_depth", 3, 10)

    model = XGBClassifier(n_estimators=n_estimators, learning_rate=learning_rate, max_depth=max_depth)
    return accuracy_score(y_test, model.fit(x_train_vec, y_train).predict(x_test_vec))

def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_xgboost, n_trials=30)

    best_params = study.best_params
    best_model = XGBClassifier(n_estimators=best_params['n_estimators'], learning_rate=best_params['learning_rate'], max_depth=best_params['max_depth'], random_state=42)

    log_mlflow("XGBoost", best_model, x_train_vec, x_test_vec, y_train, y_test)

run_optuna_experiment() 

[I 2025-08-20 22:50:18,352] A new study created in memory with name: no-name-76130bea-69d1-4227-ae14-0119433b20db
[I 2025-08-20 22:51:27,147] Trial 0 finished with value: 0.7075173720783323 and parameters: {'n_estimators': 112, 'learning_rate': 0.0006010005859381069, 'max_depth': 8}. Best is trial 0 with value: 0.7075173720783323.
[I 2025-08-20 22:51:46,335] Trial 1 finished with value: 0.6670878079595705 and parameters: {'n_estimators': 138, 'learning_rate': 0.00022822397381522356, 'max_depth': 4}. Best is trial 0 with value: 0.7075173720783323.
[I 2025-08-20 22:52:11,420] Trial 2 finished with value: 0.6673404927353127 and parameters: {'n_estimators': 175, 'learning_rate': 0.0002723254950620502, 'max_depth': 4}. Best is trial 0 with value: 0.7075173720783323.
[I 2025-08-20 22:53:32,154] Trial 3 finished with value: 0.7005685407454201 and parameters: {'n_estimators': 169, 'learning_rate': 0.000850852477543774, 'max_depth': 6}. Best is trial 0 with value: 0.7075173720783323.
[I 2025-08

🏃 View run XGBoost_TF-IDF at: http://ec2-13-61-146-35.eu-north-1.compute.amazonaws.com:5000/#/experiments/581382709887216997/runs/284aae54f2624f5d9f016eeaeeafff52
🧪 View experiment at: http://ec2-13-61-146-35.eu-north-1.compute.amazonaws.com:5000/#/experiments/581382709887216997


In [9]:
def objective_lightgbm(trial):
    n_estimators = trial.suggest_int("n_estimators", 50, 300)
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-1, log=True)
    max_depth = trial.suggest_int("max_depth", 3, 10)

    model = LGBMClassifier(n_estimators=n_estimators, learning_rate=learning_rate, max_depth=max_depth)
    return accuracy_score(y_test, model.fit(x_train_vec, y_train).predict(x_test_vec))

def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_lightgbm, n_trials=30)

    best_params = study.best_params
    best_model = LGBMClassifier(n_estimators=best_params['n_estimators'], learning_rate=best_params['learning_rate'], max_depth=best_params['max_depth'], random_state=42)

    log_mlflow("LightGBM", best_model, x_train_vec, x_test_vec, y_train, y_test)

run_optuna_experiment()

[I 2025-08-20 23:53:24,751] A new study created in memory with name: no-name-2171d51e-86d6-4079-a806-c3787120108b


[LightGBM] [Info] Number of positive: 15882, number of negative: 15777
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.351937 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 553231
[LightGBM] [Info] Number of data points in the train set: 31659, number of used features: 7956
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.501658 -> initscore=0.006633
[LightGBM] [Info] Start training from score 0.006633
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive g

c:\Users\iamha\OneDrive\Documents\SentimentAnalysis\senti\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
[I 2025-08-20 23:53:38,977] Trial 0 finished with value: 0.6937460518003791 and parameters: {'n_estimators': 119, 'learning_rate': 0.00560854756723986, 'max_depth': 4}. Best is trial 0 with value: 0.6937460518003791.


[LightGBM] [Info] Number of positive: 15882, number of negative: 15777
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.351914 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 553231
[LightGBM] [Info] Number of data points in the train set: 31659, number of used features: 7956
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.501658 -> initscore=0.006633
[LightGBM] [Info] Start training from score 0.006633


c:\Users\iamha\OneDrive\Documents\SentimentAnalysis\senti\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
[I 2025-08-20 23:53:46,354] Trial 1 finished with value: 0.8181933038534428 and parameters: {'n_estimators': 83, 'learning_rate': 0.07245309169628412, 'max_depth': 8}. Best is trial 1 with value: 0.8181933038534428.


...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Info] Number of positive: 15882, number of negative: 15777
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.397892 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 553231
[LightGBM] [Info] Number of data points in the train set: 31659, number of used features: 7956
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.501658 -> initscore=0.006633
[LightGBM] [Info] Start training from score 0.006633
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best g

c:\Users\iamha\OneDrive\Documents\SentimentAnalysis\senti\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
[I 2025-08-21 00:01:18,035] Trial 29 finished with value: 0.6969046114971573 and parameters: {'n_estimators': 120, 'learning_rate': 0.006597483173243173, 'max_depth': 4}. Best is trial 14 with value: 0.8501579279848389.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Info] Number of positive: 15882, number of negative: 15777
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.395063 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 553231
[LightGBM] [Info] Number of data points in the train set: 31659, number of used features: 7956
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.501658 -> initscore=0.006633
[LightGBM] [Info] Start training from score 0.006633
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive g

c:\Users\iamha\OneDrive\Documents\SentimentAnalysis\senti\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
2025/08/21 00:02:01 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/21 00:02:15 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run LightGBM_TF-IDF at: http://ec2-13-61-146-35.eu-north-1.compute.amazonaws.com:5000/#/experiments/581382709887216997/runs/2f68f20a75df48139e1d949b378baad5
🧪 View experiment at: http://ec2-13-61-146-35.eu-north-1.compute.amazonaws.com:5000/#/experiments/581382709887216997
